In [1]:
import pandas as pd
import requests
import shutil
import datetime
import ddddocr
from IPython.display import Image
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

session = requests.Session()

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
}


In [2]:
signal = 1
while signal:
    tmp = round(datetime.datetime.now().timestamp()*1000)
    res = session.get(f'https://portal.sw.nat.gov.tw/APGA/Captcha_validNumGenerate?time={tmp}', stream = True, verify = False)
    f = open('check.png', 'wb')
    shutil.copyfileobj(res.raw, f)
    f.close()

    ocr = ddddocr.DdddOcr()
    with open('check.png', 'rb') as f:
        img_bytes = f.read()
    code = ocr.classification(img_bytes)

    '''防呆code'''
    if len(code) == 6:
        signal = 0
        for char in code:
            if ord(char) > 57:
                signal = 1
                break

欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i


In [3]:
payload = {
    'checkCaptcha': code,
    'language': 'chinese'
}

In [4]:
res = session.post('https://portal.sw.nat.gov.tw/APGA/Captcha_checkCaptchaCode', data = payload)

In [5]:
def get_data(data_type):
    if data_type == 'import': 
        TypePort = '3'
    else:
        TypePort ='6'

    # payload = {'pStartyear': '113',
    # 'pEndyer': '113',
    # 'pStartmonth': '1',
    # 'pEndmonth': '8',
    # 'pGtNote':'',
    # 'pColSeq': '進出口別 / 日期',
    # 'pCnyList': '',
    # 'minYear': '92',
    # 'maxYear': '113',
    # 'maxMonth': '8',
    # 'minMonth': '1',
    # 'maxYearByYear': '113',
    # 'searchInfo.TypePort': TypePort,
    # 'searchInfo.TypeTime': '0',
    # 'searchInfo.StartYear': '92',
    # 'searchInfo.StartMonth': '1',
    # 'searchInfo.EndYear': '113',
    # 'searchInfo.EndMonth': '8',
    # 'searchInfo.goodsType': '0',
    # 'searchInfo.goodsName': '21',
    # 'searchInfo.goodsCodeGroup': '8207502000',
    # 'searchInfo.groupType': '1',
    # 'searchInfo.CountryName':'',
    # 'searchInfo.Type': '1',
    # 'searchInfo.OrderType': '進出口別 / 日期',
    # 'searchInfo.ExportType': 'web',
    # 'searchInfo.Captcha': code}

    payload = {'pStartyear': '92',
        'pEndyer': '113',
        'pStartmonth': '1',
        'pEndmonth': '1',
        'pGtNote': '',
        'pColSeq': '進出口別 / 日期 / 貨品別 / 國家(地區)別',
        'pCnyList': '中國大陸',
        'minYear': '92',
        'maxYear': '113',
        'maxMonth': '8',
        'minMonth': '1',
        'maxYearByYear': '113',
        'searchInfo.TypePort': TypePort,
        'searchInfo.TypeTime': '0',
        'searchInfo.StartYear': '92',
        'searchInfo.StartMonth': '1',
        'searchInfo.EndYear': '113',
        'searchInfo.EndMonth': '1',
        'searchInfo.goodsName': '21',
        'searchInfo.goodsType': '2',
        'searchInfo.goodsCodeGroup': '8207602000',
        'searchInfo.groupType': '2',
        'searchInfo.CountryName': '中國大陸',
        'searchInfo.Type': '1',
        'searchInfo.OrderType': '進出口別 / 日期 / 貨品別 / 國家(地區)別',
        'searchInfo.ExportType': 'web',
        'searchInfo.Captcha': code
    }

    res = session.post('https://portal.sw.nat.gov.tw/APGA/GA30_LIST', data = payload,  verify = True, headers = headers)
    soup = BeautifulSoup(res.text, "html.parser")
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(res.content, 'lxml')

    # Find the table that contains the monthly import/export data
    table = soup.find('table', {'id': 'dataList'})

    # Create a list to hold the data
    data = []

    # Iterate through the rows of the table and extract data
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        # print(columns)
        month_data = {
            "category": columns[0].text.strip(),
            "date": columns[1].text.strip().split('\n')[0],
            "usd_amount": columns[6].text.strip()
        }
        data.append(month_data)

    return pd.DataFrame(data)

In [6]:
data_import = get_data('import')
data_export = get_data('export')

In [7]:
test = pd.concat([data_import, data_export])

In [8]:
test.to_excel('1_進出口總值(美元).xlsx')

# 視覺化

In [9]:
df = pd.read_excel('1_進出口總值(美元).xlsx')

In [10]:
x = df[df['category'] == '出口總值(含復出口)']['date'].values
y = df[df['category'] == '出口總值(含復出口)']['usd_amount'].values